# Project Notebook
Generated from script.

In [ ]:
import sys
import os
from pathlib import Path

# Add project root to sys.path
# Assuming notebook is in 'notebooks/' and project root is one level up
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
    
from pokemon_predictor import config
from pokemon_predictor.data_utils import load_data
import pandas as pd
import numpy as np
import joblib


## 1. Load Data (RGB Features)

In [ ]:
# Load RGB Data for XGBoost
X_train_xgb, X_test_xgb, y_train, y_test, classes = load_data('rgb', split_data=True)
print(f"Classes: {classes}")
print(f"Train Shape: {X_train_xgb.shape}")


## 2. Train XGBoost Baseline

In [ ]:
from xgboost import XGBClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score

xgb = XGBClassifier(eval_metric='logloss', random_state=config.RANDOM_SEED)
clf_xgb = MultiOutputClassifier(xgb)
clf_xgb.fit(X_train_xgb, y_train)

y_pred_xgb = clf_xgb.predict(X_test_xgb)
print(f"XGBoost F1 (Macro): {f1_score(y_test, y_pred_xgb, average='macro'):.4f}")


## 3. Train MLP Baseline

In [ ]:
# Load Histogram Data for MLP
X_train_mlp, X_test_mlp, _, _, _ = load_data('hist', split_data=True)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

input_dim = X_train_mlp.shape[1]
output_dim = len(classes)

model_mlp = Sequential([
    Dense(128, activation='relu', input_shape=(input_dim,)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(output_dim, activation='sigmoid')
])

model_mlp.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model_mlp.fit(X_train_mlp, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=0)

y_pred_probs = model_mlp.predict(X_test_mlp, verbose=0)
y_pred_mlp = (y_pred_probs > 0.5).astype(int)
print(f"MLP F1 (Macro): {f1_score(y_test, y_pred_mlp, average='macro'):.4f}")
